In [1]:
import numpy as np
import pandas as pd
import cassiopeia as cass
import datetime

from csv import writer


In [2]:
cass.set_riot_api_key("RGAPI-e22a2f61-a2fb-4553-b8b8-9216f7bff4d4")
player = cass.Summoner(name="Artezzaa", region="NA")


In [3]:
def player_get_stats(lane,participant,df):

    time_5 = "05:00"
    time_10 = "10:00"
    time_15 = "15:00"

    if lane == 'top_lane':
        role = 'Top'
    elif lane == 'jungle':
        role = 'Jg'
    elif lane == 'mid_lane':
        role = 'Mid'
    elif lane == 'bot_lane':
        role = 'Bot'
    elif lane == 'utility':
        role = 'Sup'

    df['_Lane'] = lane
    df['_Champ']=participant.champion.name
    df['_Kills'] = participant.stats.kills
    df['_KDA'] = participant.stats.kda

    p_state = participant.cumulative_timeline[time_5]
    df['_Kills_5'] = p_state.kills

    p_state = participant.cumulative_timeline[time_10]
    df['_Kills_10'] = p_state.kills

    p_state = participant.cumulative_timeline[time_15]
    df['_Kills_15'] = p_state.kills

In [4]:
# Function that selects a random player from the last player's previous games
def find_next_player(history):
    game = 0
    New_Player_Attempts = 0
    while True:
        New_Player_Attempts += 1
        # the match to select a new player from
        match = history[game]
        # selecting random number to get the random player from match
        randNum = np.random.randint(11)
        player = match.participants[randNum]
        # If player has already been checked
        if player.summoner.name in already_checked:
            # only 10 players in a match, if all players been checked go to next game
            if New_Player_Attempts == 10:
                game += 1
                New_Player_Attempts = 0
                continue
            else:
                continue
        break
    print(player.summoner.name)
    # keep track of the account that has been checked
    already_checked.append(player.summoner.name)
    # Returns the puuid of the new player
    return player.summoner.puuid

In [5]:
def add_player_match_history(puuid, games):
    global player_df
    match_history = cass.MatchHistory(puuid=puuid,continent="AMERICAS", region='NA',begin_index=0,end_index=games,queue=cass.Queue.ranked_solo_fives,begin_time=int(datetime.datetime.now().timestamp() - 200))
    
    for match in match_history:
        try:
            stat_import = pd.DataFrame(columns=player_df.columns,index=[0])
        except:
            stat_import = pd.DataFrame(index=[0])
        
        if match.duration < datetime.timedelta(hours=0, minutes=15, seconds=0):
            continue
        
        p_list = []
        # Making the p objects and adding to list above
        for i in range(0,10):
            # pi = participant i
            exec('p'+str(i)+'='+'match.participants['+str(i)+']')
            # add pi to list
            exec('p_list.append(p'+str(i)+')')

        try:
            
            for p in p_list:
                if p.side.value == 100:
                    stat_import['Team_Won'] = match.blue_team.win
                    
                elif p.side.value == 200:
                    stat_import['Team_Won'] = match.red_team.win
                    
                if puuid == p.summoner.puuid:
                    if p.individual_position.name == 'top_lane':
                        player_get_stats(p.individual_position.name,p,stat_import)
                    elif p.individual_position.name == 'jungle':
                        player_get_stats(p.individual_position.name,p,stat_import)
                    elif p.individual_position.name == 'mid_lane':
                        player_get_stats(p.individual_position.name,p,stat_import)
                    elif p.individual_position.name == 'bot_lane':
                        player_get_stats(p.individual_position.name,p,stat_import)
                    elif p.individual_position.name == 'utility':
                        player_get_stats(p.individual_position.name,p,stat_import)
                    
            # stat_import.to_csv(path_or_buf="Match_Data/player_matches/matches2.csv",mode='a',index=False,header=False)
                    
            player_df = pd.concat([player_df,stat_import], ignore_index=True)
                
        except:
           pass

    return find_next_player(match_history)
    

In [6]:
i = 1
file_name = "Match_Data/player_matches/player_"+str(i-1)+"_matches.csv"
# list so we don't check the same player's games more than once
already_checked = [player.name]
# DataFrame that keeps track of all the match data over whole process
player_df = pd.DataFrame()
# Get the match info for the seed player, with this number of matches
next_puuid = add_player_match_history(player.puuid, 20)
# Print out the first block of matches from seed account with column names
player_df.to_csv(file_name,index=False,mode='w')
# Number of players you want to get matches from
while i != 5:
    file_name = "Match_Data/player_matches/player_"+str(i)+"_matches.csv"
    # empty output dataframe to stop duplicates in file
    player_df = player_df.iloc[0:0]
    # Get the match info for the randomly selected player, with this number of matches
    next_puuid = add_player_match_history(next_puuid, 20)
    # Print out the next block of matches from random selected account without columns
    player_df.to_csv(file_name,index=False,mode='w',header=True)
    i += 1

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Artezzaa
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/TbAmHuYgAZxeaiwLN-ScDFTKuksDimRmnSgLe4EsW_3CS-0HUCIE5LqTwZk-Mp2lhQZ7SPsDMUIgXw/ids?start=0&count=20&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4254552847
Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json
Making call: https://ddragon.leagueoflegends.com/api/versions.json
Making call: https://ddragon.leagueoflegends.com/cdn/12.5.1/data/en_US/championFull.json
Making call: https://ddragon.leagueoflegends.com/realms/na.json
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4254552847/timeline
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4251572080
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4251572080/timeline
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_42